## TC 5033
### Word Embeddings

<br>

#### Activity 3a: Exploring Word Embeddings with GloVe and Numpy
<br>

- Objective:
    - To understand the concept of word embeddings and their significance in Natural Language Processing.
    - To learn how to manipulate and visualize high-dimensional data using dimensionality reduction techniques like PCA and t-SNE.
    - To gain hands-on experience in implementing word similarity and analogies using GloVe embeddings and Numpy.
    
<br>

- Instructions:
    - Download GloVe pre-trained vectors from the provided link in Canvas, the official public project: 
    Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation
    https://nlp.stanford.edu/data/glove.6B.zip

    - Create a dictorionay of the embeddings so that you carry out fast look ups. Save that dictionary e.g. as a serialized file for faster loading in future uses.
    
    - PCA and t-SNE Visualization: After loading the GloVe embeddings, use Numpy and Sklearn to perform PCA and t-SNE to reduce the dimensionality of the embeddings and visualize them in a 2D or 3D space.

    - Word Similarity: Implement a function that takes a word as input and returns the 'n' most similar words based on their embeddings. You should use Numpy to implement this function, using libraries that already implement this function (e.g. Gensim) will result in zero points.

    - Word Analogies: Implement a function to solve analogies between words. For example, "man is to king as woman is to ____". You should use Numpy to implement this function, using libraries that already implement this function (e.g. Gensim) will result in zero points.

    - Submission: This activity is to be submitted in teams of 3 or 4. Only one person should submit the final work, with the full names of all team members included in a markdown cell at the beginning of the notebook.
    
<br>

- Evaluation Criteria:

    - Code Quality (40%): Your code should be well-organized, clearly commented, and easy to follow. Use also markdown cells for clarity.
    
   - Functionality (60%): All functions should work as intended, without errors.
       - Visualization of PCA and t-SNE (10% each for a total of 20%)
       - Similarity function (20%)
       - Analogy function (20%)
|



#### Import libraries

In [ ]:
# Import libraries
import torch
import torch.nn.functional as F
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
import pickle
plt.style.use('ggplot')

#### Load file

In [ ]:
# PATH = '/media/pepe/DataUbuntu/Databases/glove_embeddings/glove.6B.200d.txt'
PATH = '/media/pepe/DataUbuntu/Databases/glove_embeddings/glove.6B.50d.txt'
emb_dim = 50

In [ ]:
# Create dictionary with embeddings
def create_emb_dictionary(path):
    dict_glove={}
    with open(path, 'r', encoding='utf8') as archivo:
            for linea in archivo:
                a,*b=linea.split()
                dict_glove[a]=b
    return dict_glove

In [ ]:
# create dictionary
embeddings_dict = create_emb_dictionary(PATH)

In [ ]:
# Serialize
#with open('embeddings_dict_50D.pkl', 'wb') as f:
#    pickle.dump(embeddings_dict, f)

# Deserialize
with open('embeddings_dict_50D.pkl', 'rb') as f:
    embeddings_dict = pickle.load(f)

#### See some embeddings

In [ ]:
# Show some
def show_n_first_words(path, n_words):
        with open(path, 'r') as f:
            for i, line in enumerate(f):
                print(line.split(), len(line.split()[1:]))
                if i>=n_words: break

In [ ]:
show_n_first_words(PATH, 5)

### Plot some embeddings

In [ ]:
def plot_embeddings(emb_path, words2show, emb_dim, embeddings_dict, func = PCA):
    X=[]
    for word in words:
        X.append(embeddings_dict[word])
        
    transform=func(n_components=2)
    X_reduced=transform.fit_transform(np.asarray(X))
    plt.figure(figsize=(10,5))
    
    for i, word in enumerate(words):
        x,y = X_reduced[i]
        plt.scatter(x,y)
        plt.annotate(word,(x,y), fontsize=8)
        
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title('Word Embeddings Visualization')
    plt.grid(True)
    plt.show()

    

In [ ]:
words= ['burger', 'tortilla', 'bread', 'pizza', 'beef', 'steak', 'fries', 'chips', 
            'argentina', 'mexico', 'spain', 'usa', 'france', 'italy', 'greece', 'china',
            'water', 'beer', 'tequila', 'wine', 'whisky', 'brandy', 'vodka', 'coffee', 'tea',
            'apple', 'banana', 'orange', 'lemon', 'grapefruit', 'grape', 'strawberry', 'raspberry',
            'school', 'work', 'university', 'highschool']


In [ ]:
# 
plot_embeddings(PATH, words, emb_dim, embeddings_dict, PCA)

In [ ]:
# t-SNE dimensionality reduction for visualization
embeddings = plot_embeddings(PATH, words, emb_dim, embeddings_dict, TSNE)

### Let us compute analogies

In [ ]:
# analogy
def analogy(word1, word2, word3, embeddings_dict):
    emb_a=np.asarray(embeddings_dict[word1], dtype=np.float32)
    emb_b=np.asarray(embeddings_dict[word2], dtype=np.float32)
    emb_c=np.asarray(embeddings_dict[word3], dtype=np.float32)

    words = embeddings_dict.keys()
    max_cosine_sim = -999              
    best_word = None

    for w in words:        
        # ignore input words
        if w in [word1, word2, word3] :
            continue

        # Compute cosine similarity between the vectors u and v
        #u:(e_b - e_a) 
        #v:((w's vector representation) - e_c)
        

        u=emb_b - emb_a
        v=np.asarray(embeddings_dict[w], dtype=np.float32) - emb_c
        
        # find the dot product between u and v 
        dot = np.dot(u,v)
        # find the L2 norm of u 
        norm_u = np.sqrt(np.sum(u**2))
        # Compute the L2 norm of v
        norm_v = np.sqrt(np.sum(v**2))
        # Compute the cosine similarity
        cosine_sim = dot/(norm_u)/norm_v
        
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            # update word_d
            best_word = w
        
    return best_word

In [ ]:
analogy('man', 'king', 'woman', embeddings_dict)

In [ ]:
# most similar
def find_most_similar(word, embeddings_dict, top_n=10):
    word_tarjet=word
    similar={}
    u=np.asarray(embeddings_dict[word_tarjet], dtype=np.float32)
    
    for word in embeddings_dict.keys():     
        
        v=np.asarray(embeddings_dict[word], dtype=np.float32)
        
        # find the dot product between u and v 
        dot = np.dot(u,v)
        # find the L2 norm of u 
        norm_u = np.linalg.norm(u)
        # Compute the L2 norm of v
        norm_v = np.linalg.norm(v)
        # Compute the cosine similarity
        cosine_sim = dot/(norm_u * norm_v)
        
        similar[word] = cosine_sim
        
    sorted_similar = sorted(similar.items(), key=lambda x: x[1], reverse=True)

    return sorted_similar[1:top_n+1] 

In [ ]:
most_similar = find_most_similar('mexico', embeddings_dict)

In [ ]:
for i, w in enumerate(most_similar, 1):
    print(f'{i} ---> {w[0]}')